In [1]:
import kagglehub

# Download latest version
path = kagglehub.dataset_download("omkargurav/face-mask-dataset")

print("Path to dataset files:", path)

Path to dataset files: /kaggle/input/face-mask-dataset


In [3]:
import os
import cv2
import numpy as np
import tensorflow as tf
from glob import glob
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report

# Enable mixed precision for faster GPU training and lower memory usage
from tensorflow.keras import mixed_precision
mixed_precision.set_global_policy('mixed_float16')

# Configure GPU memory growth
gpus = tf.config.list_physical_devices('GPU')
for gpu in gpus:
    tf.config.experimental.set_memory_growth(gpu, True)

# Distributed strategy (multi-GPU) if available
tf_strategy = tf.distribute.MirroredStrategy()
print(f"GPUs detected: {tf_strategy.num_replicas_in_sync}")

# Configuration
DATA_DIR = '/kaggle/input/face-mask-dataset/data'
IMAGE_SIZE = 224
GLOBAL_BATCH_SIZE = 32  # global batch size across all GPUs
AUTO = tf.data.AUTOTUNE
EPOCHS_HEAD = 20
EPOCHS_FINE = 10
SEED = 42

# 1. Load file paths and labels
class_names = sorted(os.listdir(DATA_DIR))
paths, labels = [], []
for idx, cls in enumerate(class_names):
    cls_dir = os.path.join(DATA_DIR, cls)
    if os.path.isdir(cls_dir):
        cls_paths = glob(os.path.join(cls_dir, '*.*'))
        paths += cls_paths
        labels += [idx] * len(cls_paths)
paths = np.array(paths)
labels = np.array(labels)

# Shuffle and split
i = np.random.RandomState(SEED).permutation(len(paths))
paths, labels = paths[i], labels[i]
train_paths, test_paths, train_labels, test_labels = train_test_split(
    paths, labels, test_size=0.2, stratify=labels, random_state=SEED
)

# 2. Preprocessing function
def preprocess_image(path, label):
    img = cv2.imread(path.numpy().decode('utf-8'))
    img = cv2.resize(img, (IMAGE_SIZE, IMAGE_SIZE))
    rgb = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
    gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    eq = cv2.equalizeHist(gray)
    blur = cv2.GaussianBlur(eq, (3,3), 0)
    _, binar = cv2.threshold(blur, 0, 255, cv2.THRESH_BINARY + cv2.THRESH_OTSU)
    kernel = cv2.getStructuringElement(cv2.MORPH_RECT, (3,3))
    closed = cv2.morphologyEx(binar, cv2.MORPH_CLOSE, kernel)
    mask = closed.astype(np.float32) / 255.0

    mask3 = np.stack([mask]*3, axis=-1)
    masked = (rgb * mask3).astype(np.uint8)
    return tf.keras.applications.mobilenet_v2.preprocess_input(masked), label

# Wrap for tf.data
# Wrap for tf.data
@tf.function
def tf_preprocess(path, label):
    img, lbl = tf.py_function(preprocess_image, [path, label], [tf.float32, tf.int64]) # Changed tf.int32 to tf.int64
    img.set_shape([IMAGE_SIZE, IMAGE_SIZE, 3])
    lbl.set_shape([])
    return img, lbl

# 3. Dataset builder
with tf_strategy.scope():
    def make_dataset(paths, labels, training=True):
        ds = tf.data.Dataset.from_tensor_slices((paths, labels))
        if training:
            ds = ds.shuffle(1000, seed=SEED)
        ds = ds.map(tf_preprocess, num_parallel_calls=AUTO)
        if training:
            ds = ds.repeat()
        ds = ds.batch(GLOBAL_BATCH_SIZE)
        ds = ds.prefetch(AUTO)
        return ds

    train_ds = make_dataset(train_paths, train_labels, training=True)
    val_ds = make_dataset(test_paths, test_labels, training=False)

    # 4. Model definition with augmentation
    aug = tf.keras.Sequential([
        tf.keras.layers.RandomFlip('horizontal'),
        tf.keras.layers.RandomRotation(0.1),
        tf.keras.layers.RandomZoom(0.1),
        tf.keras.layers.RandomContrast(0.1)
    ])

    base = tf.keras.applications.MobileNetV2(
        input_shape=(IMAGE_SIZE, IMAGE_SIZE, 3), include_top=False, weights='imagenet'
    )
    base.trainable = False

    inputs = tf.keras.Input((IMAGE_SIZE, IMAGE_SIZE, 3))
    x = aug(inputs)
    x = base(x, training=False)
    x = tf.keras.layers.GlobalAveragePooling2D()(x)
    x = tf.keras.layers.Dropout(0.2)(x)
    outputs = tf.keras.layers.Dense(len(class_names), activation='softmax', dtype='float32')(x)
    model = tf.keras.Model(inputs, outputs)

    model.compile(
        optimizer=tf.keras.optimizers.Adam(),
        loss='sparse_categorical_crossentropy',
        metrics=['accuracy']
    )

# 5. Callbacks
callbacks = [
    tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=5, restore_best_weights=True),
    tf.keras.callbacks.ReduceLROnPlateau(monitor='val_loss', factor=0.2, patience=3)
]

# 6. Train head
steps_per_epoch = len(train_paths) // GLOBAL_BATCH_SIZE
history_head = model.fit(
    train_ds,
    epochs=EPOCHS_HEAD,
    steps_per_epoch=steps_per_epoch,
    validation_data=val_ds,
    validation_steps=len(test_paths) // GLOBAL_BATCH_SIZE,
    callbacks=callbacks
)

# 7. Fine-tuning last 30% of base
with tf_strategy.scope():
    total = len(base.layers)
    fine_at = int(total * 0.7)
    for layer in base.layers[fine_at:]:
        layer.trainable = True
    model.compile(
        optimizer=tf.keras.optimizers.Adam(1e-5),
        loss='sparse_categorical_crossentropy',
        metrics=['accuracy']
    )

history_fine = model.fit(
    train_ds,
    epochs=EPOCHS_HEAD + EPOCHS_FINE,
    initial_epoch=history_head.epoch[-1] + 1,
    steps_per_epoch=steps_per_epoch,
    validation_data=val_ds,
    validation_steps=len(test_paths) // GLOBAL_BATCH_SIZE,
    callbacks=callbacks
)

# 8. Final evaluation on CPU
y_preds = []
y_true = []
for imgs, lbls in val_ds.take(len(test_paths) // GLOBAL_BATCH_SIZE + 1):
    preds = np.argmax(model.predict(imgs), axis=1)
    y_preds.extend(preds)
    y_true.extend(lbls.numpy())

print(classification_report(y_true, y_preds, target_names=class_names))

GPUs detected: 1
Epoch 1/20
188/188 ━━━━━━━━━━━━━━━━━━━━ 69s 329ms/step - accuracy: 0.7540 - loss: 0.5164 - val_accuracy: 0.8152 - val_loss: 0.3857 - learning_rate: 0.0010
Epoch 2/20
188/188 ━━━━━━━━━━━━━━━━━━━━ 36s 190ms/step - accuracy: 0.8391 - loss: 0.3458 - val_accuracy: 0.8198 - val_loss: 0.3651 - learning_rate: 0.0010
Epoch 3/20
188/188 ━━━━━━━━━━━━━━━━━━━━ 35s 185ms/step - accuracy: 0.8709 - loss: 0.2928 - val_accuracy: 0.7985 - val_loss: 0.4210 - learning_rate: 0.0010
Epoch 4/20
188/188 ━━━━━━━━━━━━━━━━━━━━ 35s 189ms/step - accuracy: 0.8780 - loss: 0.2846 - val_accuracy: 0.8378 - val_loss: 0.3422 - learning_rate: 0.0010
Epoch 5/20
188/188 ━━━━━━━━━━━━━━━━━━━━ 35s 184ms/step - accuracy: 0.8685 - loss: 0.2859 - val_accuracy: 0.8072 - val_loss: 0.3833 - learning_rate: 0.0010
Epoch 6/20
188/188 ━━━━━━━━━━━━━━━━━━━━ 38s 200ms/step - accuracy: 0.8701 - loss: 0.2983 - val_accuracy: 0.7992 - val_loss: 0.4109 - learning_rate: 0.0010
Epoch 7/20
188/188 ━━━━━━━━━━━━━━━━━━━━ 41s 219ms/ste

In [4]:
# Save this model
model.save('face_mask_detection.h5')



In [6]:
# Check the path of saved model
import os
current_directory = os.getcwd()
model_path = os.path.join(current_directory, 'face_mask_detection.h5')
print(model_path)

/content/face_mask_detection.h5
